## CS5803 NLP
### Assignment 2
#### Tanmay Garg, Tanmay Goyal, Tanay Yadav
#### Roll no: CS20BTECH11063, AI20BTECH11021, AI20BTECH11026

##### Link to dataset: https://www.kaggle.com/datasets/moxxis/harry-potter-lstm.

In [1]:
# importing necessary packages
import re
from collections import defaultdict
from math import log, exp  
import numpy as np
import random
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sumit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sumit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##### **Q1. Preprocess and tokenize the dataset using NLTK**

In [2]:
# reading the original data
with open('Harry_Potter_all_char_separated.txt', 'r', encoding='utf-8') as file:
    harry_potter_data = file.read()

def preprocess_text(text):
    '''
    Function to preprocess the text by removing punctuations and converting to lower case
    '''
    # [^\w\s] -> ^ means except , \w refers to any alphanumeric character and \s refers to whitespace    
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text

def tokenize(text):
    '''
    Function to tokenize the text
    '''
    tokens = nltk.tokenize.word_tokenize(text)
    # stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokens = [token for token in tokens if token not in stop_words]
    return tokens

# preprocessing the text
harry_potter_text = preprocess_text(harry_potter_data)
# using the first 10000 words
harry_potter_tokens = tokenize(harry_potter_text)[:10000]

##### **Q2. Fit two bigram language models on the text: MLE and kneserNey Discounting**

In [3]:
# fit two bigram language models on the text: MLE and Kneser-Ney discounting using the nltk library

def MLE_bigram(n , n_gram , vocab):
    '''
    Function to fit a bigram language model using MLE
    '''
    model = nltk.lm.MLE(n)
    model.fit([n_gram], vocabulary_text = vocab)
    return model

def KN_bigram(n , n_gram , vocab):
    '''
    Function to fit a bigram language model using Kneser-Ney discounting
    '''
    model = nltk.lm.KneserNeyInterpolated(n)
    model.fit([n_gram], vocabulary_text = vocab)
    return model

harry_potter_bigrams = nltk.ngrams(harry_potter_tokens, 2)

# converting the bigrams to a list
bigrams = []
for bigram in harry_potter_bigrams:
    bigrams.append(bigram)

bigram_mle = MLE_bigram(2 , bigrams , harry_potter_tokens)
bigram_kn = KN_bigram(2 , bigrams , harry_potter_tokens)

##### **Q3. Use the beginning words 1. "Harry Potter" and 2. "Dumbledore" to generate text using both the language models. Keep maximum text length as 20**

In [4]:
def generate_prediction(model , num_words = None , text_seed = None , random_seed = None):
    '''
    Function to generate the predictions given text_seed and num_words.
    It joins them together in a sentence and returns the sentence
    '''

    # preprocessing the text_seed
    text_seed = preprocess_text(text_seed)
    text_seed = tokenize(text_seed)
    
    # generating the prediction
    pred = model.generate(num_words = num_words , text_seed = text_seed , random_seed = random_seed)
    
    sentence = text_seed + [pred[i] for i in range(num_words)]
    predicted_sentence = sentence[0]
    for word in sentence[1:]:
        predicted_sentence += ' ' + word
    return predicted_sentence


print("==== MLE Model ====")
print(generate_prediction(bigram_mle , num_words = 20 , text_seed = "Harry Potter" , random_seed = 123))
print(generate_prediction(bigram_mle , num_words = 20 , text_seed = "Dumbledore" , random_seed = 123)) 

print("==== KneserNey Model ====")
print(generate_prediction(bigram_kn , num_words = 20 , text_seed = "Harry Potter" , random_seed = 123))
print(generate_prediction(bigram_kn , num_words = 20 , text_seed = "Dumbledore" , random_seed = 123)) 

==== MLE Model ====
harry potter are are less like us all right of them angrily it he does tend to be nice if hed just
dumbledore and bacon as dudley was a man had swapped at his favorite program had expected mrs dursleys bought dudley he
==== KneserNey Model ====
harry potter are are less like us all right place you cant ive finished dialing his hair and piers and had four
dumbledore and bacon as dudley was a mad old things gray tuesday our heads down old clothes of arms and james


Beam search is a tree-based search strategy similar to BFS. In BF, we expand every child node, however, in Beam Search, we only expand the top k most probable children. The generated text is the text with the highest probabiltity

##### **Q4. To implement beam search, implement a function to find the top k most probable words**

In [5]:
def k_top_probable(model , k , text_seed):
    '''
    returns the top-k most probable words based on the model given
    ''' 
    
    # preprocessing the text_seed
    text_seed = [w.lower() for w in text_seed]
    # we store the non-zero probabilities
    non_zero_prob = {}

    for w in model.vocab:
        if model.score(w , text_seed) > 0:
            non_zero_prob[w] = model.score(w , text_seed)

    # sorting the non_zero_prob based on values
    sorted_probabilities = dict(sorted(non_zero_prob.items() , key = lambda item:item[1], reverse=True))
    if len(non_zero_prob) > k:
        return list(sorted_probabilities.keys())[:k]
    
    else:
        top = list(sorted_probabilities.keys())

        return top + [None]*(k - len(top))

In [6]:
k_top_probable(bigram_mle, 5, ["large"])

['pink', 'mustache', 'tawny', 'order', 'doughnut']

##### **Q5. Implement the Beam search using the previously trained MLE model.**

In [7]:
class BeamSearch:
    
    generated_sentences = {}
    queue = []
    
    def __init__(self , k , current_sentence , probability , depth , max_depth):
        self.k = k
        self.current_sentence = current_sentence
        self.top_probs = k_top_probable(bigram_mle, k , [self.current_sentence[-1]])
        self.children = []
        self.probability = probability
        self.depth = depth
        self.max_depth = max_depth
        BeamSearch.queue = [] if self.depth == 0 else BeamSearch.queue
        BeamSearch.generated_sentences = {} if self.depth == 0 else BeamSearch.generated_sentences
        BeamSearch.queue.append(self)

    def generate_tree(node):
        '''
        Function to generate the tree
        '''

        BeamSearch.queue = BeamSearch.queue[1:]

        if node.depth == node.max_depth:
            BeamSearch.generated_sentences[tuple(node.current_sentence)] = node.probability
            if len(BeamSearch.queue) != 0:
                BeamSearch.generate_tree(BeamSearch.queue[0])
                return
            else:
                print("Done constructing the trees")
                return

        for word in node.top_probs:
            if word is not None:
                new_sentence = node.current_sentence + [word]
                node.children.append(BeamSearch(node.k , new_sentence , node.probability * bigram_mle.score(word , [node.current_sentence[-1]]) , node.depth + 1 , node.max_depth))
        
        if len(BeamSearch.queue) != 0:
            BeamSearch.generate_tree(BeamSearch.queue[0])
        return

    def best_sentences(self , number_of_sentences):
        '''
        Function to return the best sentence
        '''
        final_sorted = sorted(BeamSearch.generated_sentences.items() , key = lambda item:item[1], reverse=True)
        print("Best sentences")
        for i in range(number_of_sentences):
            s = final_sorted[i][0]
            complete_sentence = s[0]
        
            for word in s[1:]:
                complete_sentence += ' ' + word
            print(i+1 , ". " , complete_sentence)
            print("Probability: ", final_sorted[i][1])
            print("\n")

        return final_sorted[:number_of_sentences]

#### **Q6. Implement Beam search for k=2 and depth = 10. Find the 5 generated texts with the highest probability.**

In [8]:
tree = BeamSearch(2 , ['harry'] , 1 , 0 , 10)
tree.generate_tree()
_ = tree.best_sentences(5)

Done constructing the trees
Best sentences
1 .  harry had a large pink beach ball wearing an emerald green
Probability:  4.1452323609999957e-07


2 .  harry was a large pink beach ball wearing an emerald green
Probability:  2.6321621231072855e-07


3 .  harry had a large pink beach ball wearing an emerald one
Probability:  2.0726161804999978e-07


4 .  harry was a large pink beach ball wearing an emerald one
Probability:  1.3160810615536428e-07


5 .  harry had been a large pink beach ball wearing an emerald
Probability:  9.420982638636354e-08




In [9]:
tree = BeamSearch(2 , ['dumbledore'] , 1 , 0 , 10)
tree.generate_tree()
_ = tree.best_sentences(5)

Done constructing the trees
Best sentences
1 .  dumbledore and a large pink beach ball wearing an emerald green
Probability:  1.4618533099840802e-07


2 .  dumbledore and a large pink beach ball wearing an emerald one
Probability:  7.309266549920401e-08


3 .  dumbledore you know who was a large pink beach ball wearing
Probability:  4.470384040603406e-08


4 .  dumbledore and dudley had a large pink beach ball wearing a
Probability:  4.111462434330226e-08


5 .  dumbledore you cant blame her sister marge who lived chapter the
Probability:  2.422621470240518e-08




In [10]:
class Node():

    '''
    Nodes for any tree to construct.
    Consists of the parent, state and children
    Children are stored in a list

    Methods:
    > addChild: To add a child to the node
    > isLeaf: To check if the node is a leaf
    > isRoot: To check if the node is a root
    '''
    
    def __init__(self, parent, state):
        self.parent = parent
        self.state = state
        self.children = []

    def addChild(self, child):
        self.children.append(child)

    def isLeaf(self):
        return len(self.children) == 0

    def isRoot(self):
        return self.parent is None


class BeamSearchV2(Node):

    '''
    Beam Search algorithm to generate the best N sentences based on the given model and root state

    Constructor arguments:
    > model: (any) The language model to use
    > kbestfn: (any) The function to get the k best probable words
    > root: (any) The root state to start the search
    > beam_size: (int) The beam size
    > max_depth: (int) The maximum depth to search
    > return_sentences: (default: False) Whether to return the sentences or not
    > num_best_sentences: (default: None) The number of best sentences to return

    Methods:
    > buildTree: To build the tree based on the given root and max_depth
    > getAllPaths: To get all the paths from the root to the leaf nodes
    > getNBestPaths: To get the best N paths based on the scores
    > printBestNPaths: To print the best N paths
    > printSentence: To print the sentence from the path
    '''

    def __init__(self, model, kbestfn, root, beam_size, max_depth, return_sentences=False, num_best_sentences=None):
        
        self.root = Node(parent=None, state=root)
        self.model = model
        self.beam_size = beam_size
        self.max_depth = max_depth
        self.getKBest = kbestfn
        self.pathsWithScore = {}
        self.bestNpaths = []
        self.buildTree(self.root, 0)

        
        # preforming requested tasks for sentence generation
        if return_sentences:
            if num_best_sentences is None:
                print("Provide the number of best sentences to return")
                return
            
            self.getNBestPaths(num_best_sentences)
            self.printBestNPaths()
    
    
    def buildTree(self, node, depth):

        ''' 
        Function to build the tree based on the given root and max_depth
        '''

        if depth == self.max_depth:
            return
        
        k_best = self.getKBest(self.model, self.beam_size, [node.state])

        for k in k_best:
            if k is not None:
                child = Node(parent=node, state=k)
                node.addChild(child)
                self.buildTree(child, depth+1)

    
    def getAllPaths(self, node, path):

        '''
        Function to get all the paths from the root to the leaf nodes
        '''

        if node.isLeaf():

            pathscore = 1
            
            for i in range(1, len(path)):
                pathscore *= self.model.score(path[i], [path[i-1]])
            
            self.pathsWithScore[tuple(path)] = pathscore   
            return

        for child in node.children:
            self.getAllPaths(child, path + [child.state])
    
    
    def getNBestPaths(self, N):

        '''
        Function to get the best N paths based on the scores
        '''

        self.getAllPaths(self.root, [self.root.state])
        best_paths = sorted(self.pathsWithScore.items(), key=lambda item:item[1], reverse=True)
        self.bestNpaths = best_paths[:N]

    
    def printBestNPaths(self):

        '''
        Function to print the best N paths
        '''

        for i in range(len(self.bestNpaths)):
            print("Path: ", self.printSentence(self.bestNpaths[i][0]))
            print("Score: ", self.bestNpaths[i][1])
            print("\n")

    
    def printSentence(self, pathtuple):

        '''
        Function to get the sentence from the path
        '''

        sentence = pathtuple[0]
        for word in pathtuple[1:]:
            sentence += ' ' + word
        
        return sentence

In [11]:
bsobj = BeamSearchV2(bigram_mle, k_top_probable, 'harry', 2, 10, return_sentences=True, num_best_sentences=5)

Path:  harry had a large pink beach ball wearing an emerald green
Score:  4.1452323609999957e-07


Path:  harry was a large pink beach ball wearing an emerald green
Score:  2.6321621231072855e-07


Path:  harry had a large pink beach ball wearing an emerald one
Score:  2.0726161804999978e-07


Path:  harry was a large pink beach ball wearing an emerald one
Score:  1.3160810615536428e-07


Path:  harry had been a large pink beach ball wearing an emerald
Score:  9.420982638636354e-08




In [12]:
bsobj = BeamSearchV2(bigram_mle, k_top_probable, 'dumbledore', 2, 10, return_sentences=True, num_best_sentences=5)

Path:  dumbledore and a large pink beach ball wearing an emerald green
Score:  1.4618533099840802e-07


Path:  dumbledore and a large pink beach ball wearing an emerald one
Score:  7.309266549920401e-08


Path:  dumbledore you know who was a large pink beach ball wearing
Score:  4.470384040603406e-08


Path:  dumbledore and dudley had a large pink beach ball wearing a
Score:  4.111462434330226e-08


Path:  dumbledore you cant blame her sister marge who lived chapter the
Score:  2.422621470240518e-08


